In [1]:
from glob import glob
from tqdm import tqdm
import random
import json

In [2]:
# git lfs clone https://huggingface.co/datasets/mesolitica/instructions-pair-mining
# git lfs clone https://huggingface.co/datasets/mesolitica/embedding-pair-mining

with open('train-embedding.jsonl', 'w') as fopen_l:
    
    files = glob('/home/ubuntu/instructions-pair-mining/*.jsonl')
    for f in files:
        with open(f) as fopen:
            for l in tqdm(fopen):
                
                l_ = json.loads(l)
                if not len(l_['pos']) or not len(l_['neg']):
                    continue
                if len(l_['query'].strip()) < 5:
                    continue
                
                pos = l_['pos']
                pos = [p for p in pos if len(p)]
                
                neg = l_['neg']
                neg = [p for p in neg if len(p)]
                
                l = {
                    'query': l_['query'].strip(),
                    'pos': pos,
                    'neg': neg
                }
                
                fopen_l.write(f'{json.dumps(l)}\n')
                fopen_l.flush()
                
    files = glob('/home/ubuntu/embedding-pair-mining/*.jsonl')
    files = [f for f in files if 'test' not in f]
    for f in files:
        with open(f) as fopen:
            for l in tqdm(fopen):
                l_ = json.loads(l)
                if not len(l_['pos']) or not len(l_['neg']):
                    continue
                if len(l_['query'].strip()) < 5:
                    continue
                
                if random.random() > 0.3:
                    continue
                    
                pos = l_['pos']
                pos = [p for p in pos if len(p)]
                
                neg = l_['neg']
                neg = [p for p in neg if len(p)]
                
                l = {
                    'query': l_['query'].strip(),
                    'pos': pos,
                    'neg': neg
                }
                    
                fopen_l.write(f'{json.dumps(l)}\n')
                fopen_l.flush()

15064it [00:00, 18114.06it/s]
378079it [00:15, 24131.29it/s]
77109it [00:07, 9761.66it/s] 
168647it [00:25, 6531.16it/s]
108692it [00:25, 4190.25it/s]
12779it [00:00, 17561.66it/s]
49646it [00:05, 9531.32it/s]
183842it [00:14, 13122.35it/s]
279318it [00:20, 13706.39it/s]
373944it [00:22, 16519.87it/s]
50010it [00:03, 15578.17it/s]
171047it [00:18, 9198.37it/s] 
60980it [00:03, 15990.27it/s]
169482it [00:14, 11721.63it/s]
34821it [00:04, 7593.53it/s]
3920774it [01:20, 48410.11it/s]
185318it [00:06, 29246.10it/s]
1885628it [00:37, 49982.58it/s]
425553it [00:06, 61109.96it/s]
436947it [00:18, 24155.35it/s]
1410413it [01:41, 13837.40it/s]
275460it [01:12, 3802.69it/s]
3108939it [01:35, 32561.71it/s]
103034it [00:08, 12291.42it/s]
19712it [00:02, 7303.99it/s]
8706it [00:00, 57256.76it/s]
2062574it [00:36, 56044.54it/s]
1001079it [01:04, 15490.70it/s]


In [ ]:
!shuf train-embedding.jsonl > shuf-train-embedding.jsonl

In [10]:
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
import json

class ListStr(Encoding):
    def encode(self, obj):
        return json.dumps(obj).encode()

    def decode(self, data):
        return json.loads(data)

_encodings['liststr'] = ListStr

columns = {
    'query': 'str',
    'pos': 'liststr',
    'neg': 'liststr', 
}
hashes = 'sha1', 'xxh64'

In [17]:
!rm -rf ../dataset

In [18]:
with MDSWriter(out='../dataset', columns=columns, compression=None, hashes=hashes) as out:
    with open('train-embedding.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            out.write(l)

6582248it [06:18, 17371.29it/s]


In [19]:
from streaming import LocalDataset

In [20]:
dataset = LocalDataset(local = '../dataset')

In [21]:
len(dataset)

6582248

In [23]:
dataset[-1]

{'neg': ['Ayah ada 3 orang anak dengan isteri tu dan 2 orang anak ayah ambil selepas bercerai dan yang bongsu bekas isteri ayah ambil.',
  'Father has 3 children with his wife and 2 children he took after divorce, and the youngest one was taken by his ex-wife.',
  'Ayah mempunyai 3 orang anak dengan isterinya dan 2 orang anak yang diambil selepas bercerai, dan anak bongsu diambil oleh bekas isterinya.',
  'Lepastu mak aku bagi kata-kata semangat dekat aku dan mak aku kata dia akan doakan aku. Dan nasihat mak aku, dia kata bila Allah putuskan sesuatu hubungan tu, maksudnya Allah tak redha dengan hubungan tu.',
  "My father gave me words of encouragement and my mother said she would pray for me. And my mother's advice was that when Allah ends a relationship, it means Allah is not pleased with that relationship.",
  'Bapa saya memberi kata-kata semangat kepada saya dan ibu saya berkata dia akan mendoakan saya. Dan nasihat ibu saya ialah apabila Allah mengakhiri hubungan, itu bermakna Alla